In [25]:
import json
from pyvis.network import Network

# Function to load course data from JSON and create a Pyvis graph
def create_course_graph(json_file, output_html):
    # Load the JSON data
    with open(json_file, 'r') as file:
        course_data = json.load(file)

    # Initialize the Pyvis network as a directed graph
    net = Network(notebook=False, height="750px", width="100%", directed=True)

    # Add all courses as nodes
    for course, details in course_data.items():
        net.add_node(course, label=details['title'], title=f"Course: {course}")

    # Add edges for prerequisites
    for course, details in course_data.items():
        for prereq in details.get('prerequisites', []):
            # Add missing prerequisite nodes
            if not any(node['id'] == prereq for node in net.nodes):
                net.add_node(prereq, label=f"Prerequisite: {prereq}", title=f"Prerequisite: {prereq}")
            net.add_edge(prereq, course)

    # Customize appearance
    net.repulsion(node_distance=200, spring_length=200)

    # Save the graph to an HTML file
    net.write_html(output_html)

    # Perform graph analytics: Count outgoing edges
    outgoing_edges = {node['id']: 0 for node in net.nodes}
    incoming_edges = {node['id']: 0 for node in net.nodes}

    for edge in net.edges:
        outgoing_edges[edge['from']] += 1
        incoming_edges[edge['to']] += 1

    # Sort courses by number of outgoing edges
    sorted_courses = sorted(outgoing_edges.items(), key=lambda x: x[1], reverse=True)
    print("Courses sorted by number of outgoing edges:")
    for course, count in sorted_courses:
        course_title = next((node['label'] for node in net.nodes if node['id'] == course), course)
        print(f"{course_title}: {count} outgoing edges")

    # Find the node with the most incoming edges
    max_incoming = max(incoming_edges.items(), key=lambda x: x[1])
    max_incoming_title = next((node['label'] for node in net.nodes if node['id'] == max_incoming[0]), max_incoming[0])
    print(f"Node with the most incoming edges: {max_incoming_title} ({max_incoming[1]} incoming edges)")

    return net


# Path to the JSON file
json_file = "MIT_courses.json"  # Update with the correct path if needed
output_html = "courses_graph.html"

# Generate the course graph
net = create_course_graph(json_file, output_html)

# Find the longest path in the graph

print(f"Graph saved to {output_html}. Open it in a browser to view.")



Courses sorted by number of outgoing edges:
Design and Analysis of Algorithms: 15 outgoing edges
Introduction to Probability: 15 outgoing edges
Introduction to Computer Science Programming in Python: 14 outgoing edges
Computation Structures: 14 outgoing edges
Mathematics for Computer Science: 13 outgoing edges
Prerequisite: 18.06: 13 outgoing edges
Introduction to Algorithms: 12 outgoing edges
Computer Systems Engineering: 10 outgoing edges
Software Construction: 9 outgoing edges
Prerequisite: 6.3800: 8 outgoing edges
Operating System Engineering: 7 outgoing edges
Prerequisite: 6.3900: 7 outgoing edges
Prerequisite: Permission of instructor: 6 outgoing edges
Prerequisite: 18.05: 6 outgoing edges
Prerequisite: Calculus II (GIR): 6 outgoing edges
Fundamentals of Programming: 5 outgoing edges
Prerequisite: 18.03: 5 outgoing edges
Introduction to Computational Thinking and Data Science: 4 outgoing edges
Introduction to Probability: 4 outgoing edges
Computability and Complexity Theory: 3 ou

In [24]:
# Function to find the longest path in the entire graph
def find_unique_longest_paths_in_graph(net):
    def dfs(current_node, visited, path):
        visited.add(current_node)
        path.append(current_node)

        if len(path) >= len(longest_paths[0]):
            if len(path) > len(longest_paths[0]):
                longest_paths.clear()
            if path not in longest_paths:
                longest_paths.append(list(path))

        for edge in net.edges:
            if edge['from'] == current_node and edge['to'] not in visited:
                dfs(edge['to'], visited, path)

        path.pop()
        visited.remove(current_node)

    longest_paths = [[]]

    for node in net.nodes:
        dfs(node['id'], set(), [])

    print("Unique Longest Paths in the Graph:")
    for path in longest_paths:
        print("Path:")
        for node in path:
            course_title = next((n['label'] for n in net.nodes if n['id'] == node), node)
            print(f"{node} - {course_title}")
        print()

find_unique_longest_paths_in_graph(net)

Unique Longest Paths in the Graph:
Path:
6.100A - Introduction to Computer Science Programming in Python
6.1010 - Fundamentals of Programming
6.1020 - Software Construction
6.1060 - Software Performance Engineering
6.5060 - Algorithm Engineering

Path:
6.100A - Introduction to Computer Science Programming in Python
6.1010 - Fundamentals of Programming
6.1020 - Software Construction
6.1100 - Computer Language Engineering
6.5110 - Foundations of Program Analysis

Path:
6.100A - Introduction to Computer Science Programming in Python
6.1210 - Introduction to Algorithms
6.1220 - Design and Analysis of Algorithms
6.5210 - Advanced Algorithms
6.5220 - Randomized Algorithms

Path:
6.100A - Introduction to Computer Science Programming in Python
6.1210 - Introduction to Algorithms
6.1220 - Design and Analysis of Algorithms
6.5620 - Foundations of Cryptography
6.5630 - Advanced Topics in Cryptography

Path:
6.100A - Introduction to Computer Science Programming in Python
6.1210 - Introduction to A